# Feature Engineering Script

Input variables: 45 + 1 (id)

In [3]:
# importing required packages
import pandas as pd
import pandas_profiling
import numpy as np
import os
import featuretools as ft

# To install packages, run on cmd:
# pip3 install pandas-profiling
# pip3 install featuretools

In [4]:
# Choose the city of analysis

city = 'Recife'
# city = 'CG'
# city = 'Curitiba'

In [5]:
# Put the integrated data path

integrated_data_path = os.getcwd() + "/../data/output/" + city + "/"

In [84]:
# Importing the data

output_files = []
days_of_analysis = 0

for dir_name in os.listdir(integrated_data_path):
    
    if dir_name.startswith("output_"): #to get the dir per day
        dir_path = integrated_data_path + dir_name
        days_of_analysis = days_of_analysis+1
        
        for file_name in os.listdir(dir_path):
            
            if file_name.startswith("part-"):
                file_path = dir_path + "/" + file_name
                
                # Join multiple files of the same day
                df = pd.read_csv(file_path, index_col=None, header=0)
                output_files.append(df)
                
print('Days of analysis: ', days_of_analysis)

# Join all output files            
all_integrated_data = pd.concat(output_files, axis=0, ignore_index=True)

all_integrated_data.head()

Days of analysis:  2


,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,jamBlockType,jamDelay,jamLength,jamLevel,jamSeverity,jamSpeedKM,jamDistanceToClosestShapePoint,headway,busBunching,nextBusCode
0,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50665,NaN,...,NaN,75.0,397.0,2.0,0.0,10.05,221.500244,37,False,50673
1,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50673,NaN,...,NaN,75.0,397.0,2.0,0.0,10.05,221.500244,45,False,50662
2,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50662,NaN,...,NaN,136.0,397.0,3.0,0.0,7.05,221.500244,57,False,50673
3,96,2,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50673,NaN,...,NaN,74.0,397.0,2.0,0.0,10.15,221.500244,45,False,50662
4,96,2,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50662,176265.0,...,NaN,65.0,699.0,2.0,0.0,17.55,127.828400,0,True,50663


The following variables are objetc type and should be converted to the origin types because the correlation matrix.

In [7]:
all_integrated_data.dtypes

route                                 int64
tripNum                               int64
shapeId                              object
routeFrequency                       object
shapeSequence                         int64
shapeLat                            float64
shapeLon                            float64
distanceTraveledShape               float64
busCode                               int64
gpsPointId                          float64
gpsLat                              float64
gpsLon                              float64
distanceToShapePoint                float64
gps_datetime                         object
stopPointId                           int64
problem                              object
alertDateTime                        object
alertSubtype                         object
alertType                            object
alertRoadType                       float64
alertConfidence                     float64
alertNComments                      float64
alertNImages                    

In [85]:
all_integrated_data['shapeId'] = all_integrated_data['shapeId'].astype(str)
all_integrated_data['routeFrequency'] = all_integrated_data['routeFrequency'].astype(str)
all_integrated_data['gps_datetime'] = all_integrated_data['gps_datetime'].astype('datetime64[ns]')
all_integrated_data['problem'] = all_integrated_data['problem'].astype(str)
all_integrated_data['alertDateTime'] = all_integrated_data['alertDateTime'].astype('datetime64[ns]')
all_integrated_data['alertSubtype'] = all_integrated_data['alertSubtype'].astype(str)
all_integrated_data['alertSubtype'].replace("nan", "-", inplace=True)
all_integrated_data['alertType'] = all_integrated_data['alertType'].astype(str)
all_integrated_data['alertType'].replace("nan", "-", inplace=True)
all_integrated_data['alertIsJamUnifiedAlert'] = all_integrated_data['alertIsJamUnifiedAlert'].astype(bool)
all_integrated_data['alertInScale'] = all_integrated_data['alertInScale'].astype(bool)
all_integrated_data['jamUpdateDateTime'] = all_integrated_data['jamUpdateDateTime'].astype('datetime64[ns]')
all_integrated_data['jamExpirationDateTime'] = all_integrated_data['jamExpirationDateTime'].astype('datetime64[ns]')
all_integrated_data['jamBlockType'] = all_integrated_data['jamBlockType'].astype(str)
all_integrated_data['jamBlockType'].replace("nan", "-", inplace=True)

all_integrated_data.dtypes

route                                        int64
tripNum                                      int64
shapeId                                     object
routeFrequency                              object
shapeSequence                                int64
shapeLat                                   float64
shapeLon                                   float64
distanceTraveledShape                      float64
busCode                                      int64
gpsPointId                                 float64
gpsLat                                     float64
gpsLon                                     float64
distanceToShapePoint                       float64
gps_datetime                        datetime64[ns]
stopPointId                                  int64
problem                                     object
alertDateTime                       datetime64[ns]
alertSubtype                                object
alertType                                   object
alertRoadType                  

In [86]:
# Descriptive statistics
pandas_profiling.ProfileReport(all_integrated_data, check_correlation=True, correlation_threshold=0.5)

Number of variables,45
Number of observations,180653
Total Missing (%),18.2%
Total size in memory,58.4 MiB
Average record size in memory,339.0 B
Numeric,18
Categorical,6
Boolean,2
Date,4
Text (Unique),0
Rejected,15


* Maybe it didn't measuring correlation because a lot of variables.

### 2. Creating new features


#### 2.1 With Feature Tools

In [87]:
# Creating an entity set
es = ft.EntitySet(id="all_data")

entities = es.entity_from_dataframe(entity_id="id", dataframe=all_integrated_data, make_index=True, index="id", time_index='gps_datetime')
entities

Entityset: all_data
  Entities:
    id [Rows: 180653, Columns: 46]
  Relationships:
    No relationships

In [88]:
entities.entities

[Entity: id
   Variables:
     id (dtype: index)
     route (dtype: numeric)
     tripNum (dtype: numeric)
     shapeId (dtype: categorical)
     routeFrequency (dtype: categorical)
     shapeSequence (dtype: numeric)
     shapeLat (dtype: numeric)
     shapeLon (dtype: numeric)
     distanceTraveledShape (dtype: numeric)
     busCode (dtype: numeric)
     gpsPointId (dtype: numeric)
     gpsLat (dtype: numeric)
     gpsLon (dtype: numeric)
     distanceToShapePoint (dtype: numeric)
     gps_datetime (dtype: datetime_time_index)
     stopPointId (dtype: numeric)
     problem (dtype: categorical)
     alertDateTime (dtype: datetime)
     alertSubtype (dtype: categorical)
     alertType (dtype: categorical)
     alertRoadType (dtype: numeric)
     alertConfidence (dtype: numeric)
     alertNComments (dtype: numeric)
     alertNImages (dtype: numeric)
     alertNThumbsUp (dtype: numeric)
     alertReliability (dtype: numeric)
     alertReportMood (dtype: numeric)
     alertReportRating (d

**Para a variável alvo (dados de ônibus por parada de ônibus), podemos gerar as seguintes variáveis:**

In [89]:
feature_matrix, features_defs = ft.dfs(entityset=entities, target_entity="id")

In [90]:
# Saving file
output_path = integrated_data_path + "new_feats_data.csv"
feature_matrix.to_csv(output_path, index=False)

feature_matrix.head(5)

,route,tripNum,shapeId,routeFrequency,shapeSequence,shapeLat,shapeLon,distanceTraveledShape,busCode,gpsPointId,...,YEAR(jamUpdateDateTime),YEAR(jamExpirationDateTime),MONTH(gps_datetime),MONTH(alertDateTime),MONTH(jamUpdateDateTime),MONTH(jamExpirationDateTime),WEEKDAY(gps_datetime),WEEKDAY(alertDateTime),WEEKDAY(jamUpdateDateTime),WEEKDAY(jamExpirationDateTime)
id,,,,,,,,,,,,,,,,,,,,,
0,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50665,NaN,...,2018.0,NaN,12,12.0,12.0,NaN,5,5.0,5.0,NaN
1,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50673,NaN,...,2018.0,NaN,12,12.0,12.0,NaN,5,5.0,5.0,NaN
2,96,1,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50662,NaN,...,2018.0,NaN,12,12.0,12.0,NaN,5,5.0,5.0,NaN
3,96,2,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50673,NaN,...,2018.0,NaN,12,12.0,12.0,NaN,5,5.0,5.0,NaN
4,96,2,Route_219,low_frequency,22857,-8.046589,-34.878125,3606.0,50662,176265.0,...,2018.0,NaN,12,12.0,12.0,NaN,5,5.0,5.0,NaN


In [91]:
features_defs

[<Feature: route>,
 <Feature: tripNum>,
 <Feature: shapeId>,
 <Feature: routeFrequency>,
 <Feature: shapeSequence>,
 <Feature: shapeLat>,
 <Feature: shapeLon>,
 <Feature: distanceTraveledShape>,
 <Feature: busCode>,
 <Feature: gpsPointId>,
 <Feature: gpsLat>,
 <Feature: gpsLon>,
 <Feature: distanceToShapePoint>,
 <Feature: stopPointId>,
 <Feature: problem>,
 <Feature: alertSubtype>,
 <Feature: alertType>,
 <Feature: alertRoadType>,
 <Feature: alertConfidence>,
 <Feature: alertNComments>,
 <Feature: alertNImages>,
 <Feature: alertNThumbsUp>,
 <Feature: alertReliability>,
 <Feature: alertReportMood>,
 <Feature: alertReportRating>,
 <Feature: alertSpeed>,
 <Feature: alertLatitude>,
 <Feature: alertLongitude>,
 <Feature: alertDistanceToClosestShapePoint>,
 <Feature: alertIsJamUnifiedAlert>,
 <Feature: alertInScale>,
 <Feature: jamBlockType>,
 <Feature: jamDelay>,
 <Feature: jamLength>,
 <Feature: jamLevel>,
 <Feature: jamSeverity>,
 <Feature: jamSpeedKM>,
 <Feature: jamDistanceToClosestSha

About feature types:
* It works with data type, but generate just mean, sum, count, mode...